### Train a joint LSTM on all signals simultaneously

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "3"
from phase.embedding import *

PATH = os.path.expanduser("~/phase/upstream_embedding/")

for hosp_data in [0,1]:
    DATAPATH  = '{}data/min5_data/{}minimum5/hospital_{}/proc/'.format(PATH,"SAO2",hosp_data)
    task = "nextfive"
    # task = "minimum5"
    # task = "hypo"

    ########## Form Data #########
    X_train_lst = []; X_valid_lst = []
    y_train_lst = []; y_valid_lst = []

    for i in range(0,len(X_lst)):
        feat = X_lst[i]

        data = load_train_val_data(DATAPATH,task,feat=feat,save_proc_y=True,
                                   filter_zeros=False,impute_per_sample=False)
        X_train, y_train, X_valid, y_valid = data

        X_train_lst.append(X_train)
        X_valid_lst.append(X_valid)
        y_train_lst.append(y_train)
        y_valid_lst.append(y_valid)
        
    opt_name="rmsprop";lr=0.001;drop=0.5
    b_size=1000;epoch_num=200;nodesize=200
    per_epoch_size = 300000

    # Fixed hyperpara
    print("[PROGRESS] Starting create_train_model()")
    h1 = nodesize; h2 = nodesize
    lookback = 60; loss_func = "mse"

    # Set opt based on opt_name
    if opt_name is "rmsprop":
        opt = keras.optimizers.RMSprop(lr)
    elif opt_name is "sgd":
        opt = keras.optimizers.SGD(lr)
    elif opt_name is "adam":
        opt = keras.optimizers.Adam(lr)

    # Form the model name (for saving the model)
    mod_name  = "multivariate_biglstmdropout_hd{}_".format(hosp_data)
    mod_name += "{}task_{}n_{}n_{}ep".format(task,h1,h2,epoch_num)
    mod_name += "_{}opt_{}lr".format(opt_name,lr)
    mod_name += "_{}drop_{}bs_epochsize".format(drop,b_size,per_epoch_size)
    MODDIR = PATH+"models/"+mod_name+"/"

    loss_keys = ["loss", 'val_loss']
    [loss_keys.append("val_dense_{}_loss".format(i)) for i in range(1,16)]

    if not os.path.exists(MODDIR): 
        os.makedirs(MODDIR)
        with open(MODDIR+"loss.txt", "w") as f:
            f.write("\t".join(["i", "epoch_time"] + loss_keys)+"\n")

        ########## Form Model #########
        sig_lst     = []; encoded_lst = []
        for i in range(0,len(X_lst)):
            sig = Input(shape=(lookback,1))
            lstm1 = LSTM(h1, recurrent_dropout=drop, return_sequences=True)
            lstm2 = LSTM(h2, recurrent_dropout=drop, dropout=drop)

            encoded = lstm2(lstm1(sig))
            sig_lst.append(sig); encoded_lst.append(encoded)
        merged_vector = keras.layers.concatenate(encoded_lst, axis=-1)

        prediction_lst = []
        for i in range(0,len(X_lst)):
            prediction = Dense(5, activation='sigmoid')(merged_vector)
            prediction_lst.append(prediction)

        model = Model(inputs=sig_lst, outputs=prediction_lst)
        model = multi_gpu_model(model,gpus=GPUNUM)
        model.compile(optimizer=opt, loss=loss_func)

        ########## Training #########
        print("[PROGRESS] Starting train_model()")

        # Train and Save
        start_time = time.time()
        for i in range(0,epoch_num):
            train_subset_inds = np.random.choice(X_train_lst[0].shape[0],per_epoch_size,replace=False)
            X_train_lst_sub = [X[train_subset_inds] for X in X_train_lst]
            y_train_lst_sub = [y[train_subset_inds] for y in y_train_lst]
            history = model.fit(X_train_lst_sub, y_train_lst_sub, epochs=1, batch_size=b_size, 
                                validation_data=(X_valid_lst,y_valid_lst))

            # Save details about training
            epoch_time = time.time() - start_time
            write_lst = [i, epoch_time]
            write_lst += [history.history[k][0] for k in loss_keys]
            with open(MODDIR+"loss.txt", "a") as f:
                f.write("\t".join([str(round(e,5)) for e in write_lst])+"\n")

            # Save model each iteration
            val_loss = history.history['val_loss'][0]
            model.save("{}val_loss:{}_epoch:{}_{}.h5".format(MODDIR,val_loss,i,mod_name))

Using TensorFlow backend.
W0929 17:26:25.804311 139970483787584 deprecation_wrapper.py:119] From ../embedding_utils.py:16: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.

W0929 17:26:25.806766 139970483787584 deprecation_wrapper.py:119] From ../embedding_utils.py:16: The name tf.GPUOptions is deprecated. Please use tf.compat.v1.GPUOptions instead.

W0929 17:26:25.807962 139970483787584 deprecation_wrapper.py:119] From ../embedding_utils.py:17: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



[PROGRESS] Starting load_train_val_data()
[DEBUG] Loading from DATAPATH: /homes/gws/hughchen/phase/upstream_embedding/data/min5_data/SAO2minimum5/hospital_1/proc/
[DEBUG] Loading train_x_file: X_train_60_SAO2minimum5_featnum:2_featname:ECGRATE.npy
[DEBUG] Loading val_x_file  : X_val_60_SAO2minimum5_featnum:2_featname:ECGRATE.npy
[DEBUG] Loading train_y_file: y_train_60_ECGRATEnextfive.npy
[DEBUG] Loading val_y_file  : y_val_60_ECGRATEnextfive.npy
[DEBUG] Saving processed y files
[DEBUG] train_x.shape: (3785233, 60, 1), train_y.shape: (3785233, 5)
[DEBUG] val_x.shape  : (420582, 60, 1),   val_y.shape: (420582, 5)
[PROGRESS] Starting load_train_val_data()
[DEBUG] Loading from DATAPATH: /homes/gws/hughchen/phase/upstream_embedding/data/min5_data/SAO2minimum5/hospital_1/proc/
[DEBUG] Loading train_x_file: X_train_60_SAO2minimum5_featnum:3_featname:ETCO2.npy
[DEBUG] Loading val_x_file  : X_val_60_SAO2minimum5_featnum:3_featname:ETCO2.npy
[DEBUG] Loading train_y_file: y_train_60_ETCO2nextfiv